**Numerical Methods Coursework 1**

In [ ]:
import numpy as np

Firstly we implement the Kamrad-Ritchken model.

In [ ]:

class TrinomialTreeKamradRitchken():
    def __init__(self, r, dt, sigma, s0, N, lamda):
        #model parameters
        self._dt = dt
        self._r = r
        self._u=np.exp(lamda*sigma*np.sqrt(dt))
        self._qu = 1/2#1/(2*lamda**2) + ((r - sigma**2/2)*np.sqrt(dt))/2*lamda*sigma
        self._qm = 1-1/lamda**2
        self._qd = 1/2#1/(2*lamda**2) - ((r - sigma**2/2)*np.sqrt(dt))/2*lamda*sigma

        if (self._qu < 0 or self._qd < 0):
            print("The value of dt is too large and the probabilities will not be viable, please use a smaller value for dt.")
            return

        # Set up the tree via S[k,n] = s0 * u^(2k-n)
        self._prices = np.zeros((2*N+1, N+1))
        for t in range(N+1):
            self._prices[0:2*t+1, t] = s0 * self._u**np.arange(-t, t+1, 1)

    def get_stock_price(self, n,k):
        return self._prices[k,n]

    def discount(self, x, number_of_periods = 1):
        return x*np.exp(-1*self._r*self._dt*number_of_periods)

    def get_u(self):
        return self._u

    def get_probabilities(self):
        return np.array([self._qd, self._qm, self._qu])
        

We create an array of prices for $A^n_j$ noticing that these prices have no dependence on $n$.

In [ ]:
def create_auxillary_values(s0, N, u):
    indices = np.arange(0,N+1)
    AValues = s0*u**indices
    return AValues

Next we define the funtion for generating the prices by backward induction

In [ ]:
def generate_lookback_prices(s0, r, dt, sigma, N, lamda, american = False):
               
    model = TrinomialTreeKamradRitchken(r, dt, sigma, s0, N, lamda)
    u = model.get_u()
    auxillary_values = create_auxillary_values(s0, N, u)
            
    # Option values matrix indexed by time, k, j
    option_values = np.zeros([N+1, 2*N+1, N+1])

    # Terminal condition: option values given by terminal payoff
    for k in range(2*N+1):
        stock_price = model.get_stock_price(N,k)
        option_values[N, k, :] = auxillary_values - stock_price
            
    # Backward induction

    probabilities = model.get_probabilities()

    # Loop in time
    for n in reversed(range(N)):
        
        
        #generate the possible values of k_new indexed by current k, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        k_new = np.column_stack([np.arange(0, 2*n+1), np.arange(1,2*n+2), np.arange(2,2*n+3)])

        j_range = np.arange(0,n+1)

        #Use the shooting function to generate the possible values of jnew indexed by current k, current j, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        phi_result = np.maximum(j_range[np.newaxis,:,np.newaxis], k_new[:,np.newaxis,:] -n -1)

        #calculate the option prices using backward induction
        
        #restrict the option_values to just legal values of k at time n+1
        option_values_restricted = option_values[n+1, 0:phi_result.shape[0], :]

        current_time_values = (probabilities * np.take_along_axis(option_values_restricted[:,:,np.newaxis], phi_result, axis = 1)).sum(axis=2)
        
        option_values[n][0:2*n+1,0:n+1] =  model.discount(current_time_values)


            
    
    return option_values

*Finally let's run the code to obtain the price

In [ ]:
s0 = 100
r = 0
dt = 1
sigma = 0.1
N = 2
lamda = 1
generate_lookback_prices(s0, r, dt, sigma, N, lamda, american=False)